In [2]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline


In [3]:
seed = 7
np.random.seed(seed)

In [7]:
# loading the dataset already downloaed in route folder
df = pd.read_csv("./sonar.csv", header=None)
df.head()

,0,1,2,3,4,5,6,7,8,9,...,51,52,53,54,55,56,57,58,59,60
0,0.0200,0.0371,0.0428,0.0207,0.0954,0.0986,0.1539,0.1601,0.3109,0.2111,...,0.0027,0.0065,0.0159,0.0072,0.0167,0.0180,0.0084,0.0090,0.0032,R
1,0.0453,0.0523,0.0843,0.0689,0.1183,0.2583,0.2156,0.3481,0.3337,0.2872,...,0.0084,0.0089,0.0048,0.0094,0.0191,0.0140,0.0049,0.0052,0.0044,R
2,0.0262,0.0582,0.1099,0.1083,0.0974,0.2280,0.2431,0.3771,0.5598,0.6194,...,0.0232,0.0166,0.0095,0.0180,0.0244,0.0316,0.0164,0.0095,0.0078,R
3,0.0100,0.0171,0.0623,0.0205,0.0205,0.0368,0.1098,0.1276,0.0598,0.1264,...,0.0121,0.0036,0.0150,0.0085,0.0073,0.0050,0.0044,0.0040,0.0117,R
4,0.0762,0.0666,0.0481,0.0394,0.0590,0.0649,0.1209,0.2467,0.3564,0.4459,...,0.0031,0.0054,0.0105,0.0110,0.0015,0.0072,0.0048,0.0107,0.0094,R


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 208 entries, 0 to 207
Data columns (total 61 columns):
0     208 non-null float64
1     208 non-null float64
2     208 non-null float64
3     208 non-null float64
4     208 non-null float64
5     208 non-null float64
6     208 non-null float64
7     208 non-null float64
8     208 non-null float64
9     208 non-null float64
10    208 non-null float64
11    208 non-null float64
12    208 non-null float64
13    208 non-null float64
14    208 non-null float64
15    208 non-null float64
16    208 non-null float64
17    208 non-null float64
18    208 non-null float64
19    208 non-null float64
20    208 non-null float64
21    208 non-null float64
22    208 non-null float64
23    208 non-null float64
24    208 non-null float64
25    208 non-null float64
26    208 non-null float64
27    208 non-null float64
28    208 non-null float64
29    208 non-null float64
30    208 non-null float64
31    208 non-null float64
32    208 non-null float64
33  

In [41]:
# limit to categorical data using df.select_dtypes()
x=df.select_dtypes(include=[np.number])
y=df.select_dtypes(include=[object])

In [33]:
x.tail()  # All Inputs Parameters

,0,1,2,3,4,5,6,7,8,9,...,50,51,52,53,54,55,56,57,58,59
203,0.0187,0.0346,0.0168,0.0177,0.0393,0.1630,0.2028,0.1694,0.2328,0.2684,...,0.0203,0.0116,0.0098,0.0199,0.0033,0.0101,0.0065,0.0115,0.0193,0.0157
204,0.0323,0.0101,0.0298,0.0564,0.0760,0.0958,0.0990,0.1018,0.1030,0.2154,...,0.0051,0.0061,0.0093,0.0135,0.0063,0.0063,0.0034,0.0032,0.0062,0.0067
205,0.0522,0.0437,0.0180,0.0292,0.0351,0.1171,0.1257,0.1178,0.1258,0.2529,...,0.0155,0.0160,0.0029,0.0051,0.0062,0.0089,0.0140,0.0138,0.0077,0.0031
206,0.0303,0.0353,0.0490,0.0608,0.0167,0.1354,0.1465,0.1123,0.1945,0.2354,...,0.0042,0.0086,0.0046,0.0126,0.0036,0.0035,0.0034,0.0079,0.0036,0.0048
207,0.0260,0.0363,0.0136,0.0272,0.0214,0.0338,0.0655,0.1400,0.1843,0.2354,...,0.0181,0.0146,0.0129,0.0047,0.0039,0.0061,0.0040,0.0036,0.0061,0.0115


In [44]:
x.shape

(208, 60)

In [42]:
y.tail()  # All Output labels

,60
203,M
204,M
205,M
206,M
207,M


In [45]:
le = LabelEncoder()
# 2/3. FIT AND TRANSFORM
# use df.apply() to apply le.fit_transform to all column values
y = y.apply(le.fit_transform)

In [46]:
# baseline model
def create_baseline():
    
   # create model, write code below
    model =Sequential()
    model.add(Dense(60, activation='relu', input_shape=(60,)))
    model.add(Dense(1, activation='sigmoid'))

  # Compile model, write code below
    model.compile(optimizer='Adam',loss='binary_crossentropy',metrics=['accuracy'])
    return model

In [47]:
# evaluate model with standardized dataset
estimator = KerasClassifier(build_fn=create_baseline, epochs=100, batch_size=5, verbose=1)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(estimator, x, y, cv=kfold)
print("Results: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Epoch 1/100
186/186 [==============================] - 1s 7ms/step - loss: 0.6745 - acc: 0.5591
Epoch 2/100
186/186 [==============================] - 0s 592us/step - loss: 0.6437 - acc: 0.6720
Epoch 3/100
186/186 [==============================] - 0s 543us/step - loss: 0.6166 - acc: 0.7204
Epoch 4/100
186/186 [==============================] - 0s 511us/step - loss: 0.5829 - acc: 0.7312
Epoch 5/100
186/186 [==============================] - 0s 513us/step - loss: 0.5797 - acc: 0.6935
Epoch 6/100
186/186 [==============================] - 0s 527us/step - loss: 0.5484 - acc: 0.7581
Epoch 7/100
186/186 [==============================] - 0s 532us/step - loss: 0.5314 - acc: 0.7957
Epoch 8/100
186/186 [==============================] - 0s 476us/step - loss: 0.5169 - acc: 0.7903
Epoch 9/100
186/186 [==============================] - 0s 446us/step - loss: 0.5185 - acc: 0.7581
Epoch 10/100
186/186 [==============================] - 0s 468us/step - loss: 0.4949 - acc: 0.8065
Epoch 11/100
186/186 

186/186 [==============================] - 0s 231us/step - loss: 0.1882 - acc: 0.9462
Epoch 84/100
186/186 [==============================] - 0s 231us/step - loss: 0.1893 - acc: 0.9301
Epoch 85/100
186/186 [==============================] - 0s 237us/step - loss: 0.1883 - acc: 0.9462
Epoch 86/100
186/186 [==============================] - 0s 231us/step - loss: 0.1883 - acc: 0.9570
Epoch 87/100
186/186 [==============================] - 0s 247us/step - loss: 0.1805 - acc: 0.9355
Epoch 88/100
186/186 [==============================] - 0s 226us/step - loss: 0.1871 - acc: 0.9462
Epoch 89/100
186/186 [==============================] - 0s 242us/step - loss: 0.1731 - acc: 0.9570
Epoch 90/100
186/186 [==============================] - 0s 247us/step - loss: 0.1654 - acc: 0.9677
Epoch 91/100
186/186 [==============================] - 0s 258us/step - loss: 0.1729 - acc: 0.9570
Epoch 92/100
186/186 [==============================] - 0s 237us/step - loss: 0.1766 - acc: 0.9516
Epoch 93/100
186/186 [=

187/187 [==============================] - 0s 300us/step - loss: 0.2335 - acc: 0.9305
Epoch 66/100
187/187 [==============================] - 0s 284us/step - loss: 0.2328 - acc: 0.9305
Epoch 67/100
187/187 [==============================] - 0s 238us/step - loss: 0.2290 - acc: 0.9251
Epoch 68/100
187/187 [==============================] - 0s 261us/step - loss: 0.2220 - acc: 0.9358
Epoch 69/100
187/187 [==============================] - 0s 271us/step - loss: 0.2258 - acc: 0.9305
Epoch 70/100
187/187 [==============================] - 0s 231us/step - loss: 0.2200 - acc: 0.9251
Epoch 71/100
187/187 [==============================] - 0s 257us/step - loss: 0.2168 - acc: 0.9305
Epoch 72/100
187/187 [==============================] - 0s 251us/step - loss: 0.2244 - acc: 0.9144
Epoch 73/100
187/187 [==============================] - 0s 251us/step - loss: 0.2236 - acc: 0.9198
Epoch 74/100
187/187 [==============================] - 0s 241us/step - loss: 0.2136 - acc: 0.9144
Epoch 75/100
187/187 [=

187/187 [==============================] - 0s 316us/step - loss: 0.2906 - acc: 0.8663
Epoch 48/100
187/187 [==============================] - 0s 316us/step - loss: 0.2866 - acc: 0.8984
Epoch 49/100
187/187 [==============================] - 0s 303us/step - loss: 0.2836 - acc: 0.8930
Epoch 50/100
187/187 [==============================] - 0s 284us/step - loss: 0.2783 - acc: 0.9144
Epoch 51/100
187/187 [==============================] - 0s 272us/step - loss: 0.2790 - acc: 0.8984
Epoch 52/100
187/187 [==============================] - 0s 310us/step - loss: 0.2727 - acc: 0.8984
Epoch 53/100
187/187 [==============================] - 0s 332us/step - loss: 0.2703 - acc: 0.9091
Epoch 54/100
187/187 [==============================] - 0s 310us/step - loss: 0.2674 - acc: 0.8984
Epoch 55/100
187/187 [==============================] - 0s 332us/step - loss: 0.2634 - acc: 0.8770
Epoch 56/100
187/187 [==============================] - 0s 332us/step - loss: 0.2703 - acc: 0.8930
Epoch 57/100
187/187 [=

187/187 [==============================] - 0s 358us/step - loss: 0.3546 - acc: 0.8503
Epoch 30/100
187/187 [==============================] - 0s 369us/step - loss: 0.3625 - acc: 0.8342
Epoch 31/100
187/187 [==============================] - 0s 428us/step - loss: 0.3540 - acc: 0.8503
Epoch 32/100
187/187 [==============================] - 0s 369us/step - loss: 0.3468 - acc: 0.8556
Epoch 33/100
187/187 [==============================] - 0s 247us/step - loss: 0.3391 - acc: 0.8663
Epoch 34/100
187/187 [==============================] - 0s 382us/step - loss: 0.3300 - acc: 0.8717
Epoch 35/100
187/187 [==============================] - 0s 237us/step - loss: 0.3432 - acc: 0.8449
Epoch 36/100
187/187 [==============================] - 0s 399us/step - loss: 0.3280 - acc: 0.8824
Epoch 37/100
187/187 [==============================] - 0s 326us/step - loss: 0.3314 - acc: 0.8770
Epoch 38/100
187/187 [==============================] - 0s 248us/step - loss: 0.3283 - acc: 0.8877
Epoch 39/100
187/187 [=

...

187/187 [==============================] - 0s 305us/step - loss: 0.2760 - acc: 0.8984
Epoch 58/100
187/187 [==============================] - 0s 273us/step - loss: 0.2693 - acc: 0.8877
Epoch 59/100
187/187 [==============================] - 0s 267us/step - loss: 0.2668 - acc: 0.8930
Epoch 60/100
187/187 [==============================] - 0s 257us/step - loss: 0.2725 - acc: 0.9144
Epoch 61/100
187/187 [==============================] - 0s 316us/step - loss: 0.2603 - acc: 0.8930
Epoch 62/100
187/187 [==============================] - 0s 284us/step - loss: 0.2532 - acc: 0.9091
Epoch 63/100
187/187 [==============================] - 0s 237us/step - loss: 0.2765 - acc: 0.8984
Epoch 64/100
187/187 [==============================] - 0s 266us/step - loss: 0.2571 - acc: 0.9037
Epoch 65/100
187/187 [==============================] - 0s 278us/step - loss: 0.2496 - acc: 0.8930
Epoch 66/100
187/187 [==============================] - 0s 259us/step - loss: 0.2612 - acc: 0.9198
Epoch 67/100
187/187 [=

188/188 [==============================] - 0s 378us/step - loss: 0.3031 - acc: 0.8511
Epoch 40/100
188/188 [==============================] - 0s 357us/step - loss: 0.2950 - acc: 0.8989
Epoch 41/100
188/188 [==============================] - 0s 351us/step - loss: 0.2916 - acc: 0.8936
Epoch 42/100
188/188 [==============================] - 0s 343us/step - loss: 0.2940 - acc: 0.8617
Epoch 43/100
188/188 [==============================] - 0s 232us/step - loss: 0.2973 - acc: 0.8670
Epoch 44/100
188/188 [==============================] - 0s 428us/step - loss: 0.2915 - acc: 0.8670
Epoch 45/100
188/188 [==============================] - 0s 365us/step - loss: 0.2812 - acc: 0.8830
Epoch 46/100
188/188 [==============================] - 0s 378us/step - loss: 0.2756 - acc: 0.9096
Epoch 47/100
188/188 [==============================] - 0s 357us/step - loss: 0.2956 - acc: 0.8670
Epoch 48/100
188/188 [==============================] - 0s 362us/step - loss: 0.2680 - acc: 0.8936
Epoch 49/100
188/188 [=

188/188 [==============================] - 0s 447us/step - loss: 0.3997 - acc: 0.8298
Epoch 22/100
188/188 [==============================] - 0s 447us/step - loss: 0.3834 - acc: 0.8404
Epoch 23/100
188/188 [==============================] - 0s 496us/step - loss: 0.3668 - acc: 0.8457
Epoch 24/100
188/188 [==============================] - 0s 474us/step - loss: 0.3654 - acc: 0.8351
Epoch 25/100
188/188 [==============================] - 0s 385us/step - loss: 0.3628 - acc: 0.8404
Epoch 26/100
188/188 [==============================] - 0s 404us/step - loss: 0.3553 - acc: 0.8404
Epoch 27/100
188/188 [==============================] - 0s 394us/step - loss: 0.3477 - acc: 0.8617
Epoch 28/100
188/188 [==============================] - 0s 415us/step - loss: 0.3460 - acc: 0.8564
Epoch 29/100
188/188 [==============================] - 0s 378us/step - loss: 0.3347 - acc: 0.8670
Epoch 30/100
188/188 [==============================] - 0s 382us/step - loss: 0.3260 - acc: 0.8511
Epoch 31/100
188/188 [=

188/188 [==============================] - 0s 511us/step - loss: 0.6324 - acc: 0.6915
Epoch 4/100
188/188 [==============================] - 0s 541us/step - loss: 0.6041 - acc: 0.7926
Epoch 5/100
188/188 [==============================] - 0s 633us/step - loss: 0.5769 - acc: 0.7926
Epoch 6/100
188/188 [==============================] - 0s 543us/step - loss: 0.5662 - acc: 0.7500
Epoch 7/100
188/188 [==============================] - 0s 500us/step - loss: 0.5367 - acc: 0.7660
Epoch 8/100
188/188 [==============================] - 0s 442us/step - loss: 0.5428 - acc: 0.7553
Epoch 9/100
188/188 [==============================] - 0s 490us/step - loss: 0.5021 - acc: 0.8298
Epoch 10/100
188/188 [==============================] - 0s 458us/step - loss: 0.4882 - acc: 0.8085
Epoch 11/100
188/188 [==============================] - 0s 458us/step - loss: 0.4870 - acc: 0.7713
Epoch 12/100
188/188 [==============================] - 0s 436us/step - loss: 0.4592 - acc: 0.8191
Epoch 13/100
188/188 [=======

188/188 [==============================] - 0s 234us/step - loss: 0.1487 - acc: 0.9628
Epoch 86/100
188/188 [==============================] - 0s 234us/step - loss: 0.1437 - acc: 0.9734
Epoch 87/100
188/188 [==============================] - 0s 229us/step - loss: 0.1434 - acc: 0.9681
Epoch 88/100
188/188 [==============================] - 0s 239us/step - loss: 0.1437 - acc: 0.9628
Epoch 89/100
188/188 [==============================] - 0s 239us/step - loss: 0.1445 - acc: 0.9574
Epoch 90/100
188/188 [==============================] - 0s 239us/step - loss: 0.1395 - acc: 0.9787
Epoch 91/100
188/188 [==============================] - 0s 204us/step - loss: 0.1376 - acc: 0.9628
Epoch 92/100
188/188 [==============================] - 0s 244us/step - loss: 0.1358 - acc: 0.9681
Epoch 93/100
188/188 [==============================] - 0s 217us/step - loss: 0.1363 - acc: 0.9734
Epoch 94/100
188/188 [==============================] - 0s 277us/step - loss: 0.1401 - acc: 0.9734
Epoch 95/100
188/188 [=

In [48]:
# evaluate baseline model with standardized dataset
np.random.seed(seed)
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasClassifier(build_fn=create_baseline, epochs=100, batch_size=5, verbose=1)))
pipeline = Pipeline(estimators)

kfold = StratifiedKFold(n_splits=15, shuffle=True, random_state=seed)
results = cross_val_score(pipeline, x, y, cv=kfold)
print("Standardized: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))


Epoch 1/100
193/193 [==============================] - 1s 6ms/step - loss: 0.5528 - acc: 0.7047
Epoch 2/100
193/193 [==============================] - 0s 555us/step - loss: 0.4160 - acc: 0.8135
Epoch 3/100
193/193 [==============================] - 0s 565us/step - loss: 0.3390 - acc: 0.8756
Epoch 4/100
193/193 [==============================] - 0s 591us/step - loss: 0.2926 - acc: 0.9171
Epoch 5/100
193/193 [==============================] - 0s 653us/step - loss: 0.2524 - acc: 0.9378
Epoch 6/100
193/193 [==============================] - 0s 513us/step - loss: 0.2195 - acc: 0.9430
Epoch 7/100
193/193 [==============================] - 0s 503us/step - loss: 0.1942 - acc: 0.9534
Epoch 8/100
193/193 [==============================] - 0s 498us/step - loss: 0.1716 - acc: 0.9741
Epoch 9/100
193/193 [==============================] - 0s 508us/step - loss: 0.1512 - acc: 0.9845
Epoch 10/100
193/193 [==============================] - 0s 482us/step - loss: 0.1340 - acc: 0.9896
Epoch 11/100
193/193 

193/193 [==============================] - 0s 259us/step - loss: 0.0015 - acc: 1.0000
Epoch 84/100
193/193 [==============================] - 0s 264us/step - loss: 0.0015 - acc: 1.0000
Epoch 85/100
193/193 [==============================] - 0s 254us/step - loss: 0.0014 - acc: 1.0000
Epoch 86/100
193/193 [==============================] - 0s 198us/step - loss: 0.0014 - acc: 1.0000
Epoch 87/100
193/193 [==============================] - 0s 252us/step - loss: 0.0013 - acc: 1.0000
Epoch 88/100
193/193 [==============================] - 0s 229us/step - loss: 0.0013 - acc: 1.0000
Epoch 89/100
193/193 [==============================] - 0s 204us/step - loss: 0.0013 - acc: 1.0000
Epoch 90/100
193/193 [==============================] - 0s 259us/step - loss: 0.0012 - acc: 1.0000
Epoch 91/100
193/193 [==============================] - 0s 237us/step - loss: 0.0012 - acc: 1.0000
Epoch 92/100
193/193 [==============================] - 0s 195us/step - loss: 0.0011 - acc: 1.0000
Epoch 93/100
193/193 [=

193/193 [==============================] - 0s 238us/step - loss: 0.0031 - acc: 1.0000
Epoch 66/100
193/193 [==============================] - 0s 249us/step - loss: 0.0030 - acc: 1.0000
Epoch 67/100
193/193 [==============================] - 0s 247us/step - loss: 0.0028 - acc: 1.0000
Epoch 68/100
193/193 [==============================] - 0s 298us/step - loss: 0.0028 - acc: 1.0000
Epoch 69/100
193/193 [==============================] - 0s 219us/step - loss: 0.0026 - acc: 1.0000
Epoch 70/100
193/193 [==============================] - 0s 258us/step - loss: 0.0025 - acc: 1.0000
Epoch 71/100
193/193 [==============================] - 0s 321us/step - loss: 0.0024 - acc: 1.0000
Epoch 72/100
193/193 [==============================] - 0s 264us/step - loss: 0.0024 - acc: 1.0000
Epoch 73/100
193/193 [==============================] - 0s 235us/step - loss: 0.0023 - acc: 1.0000
Epoch 74/100
193/193 [==============================] - 0s 238us/step - loss: 0.0022 - acc: 1.0000
Epoch 75/100
193/193 [=

193/193 [==============================] - 0s 321us/step - loss: 0.0083 - acc: 1.0000
Epoch 48/100
193/193 [==============================] - 0s 378us/step - loss: 0.0079 - acc: 1.0000
Epoch 49/100
193/193 [==============================] - 0s 337us/step - loss: 0.0074 - acc: 1.0000
Epoch 50/100
193/193 [==============================] - 0s 256us/step - loss: 0.0071 - acc: 1.0000
Epoch 51/100
193/193 [==============================] - 0s 433us/step - loss: 0.0068 - acc: 1.0000
Epoch 52/100
193/193 [==============================] - 0s 255us/step - loss: 0.0064 - acc: 1.0000
Epoch 53/100
193/193 [==============================] - 0s 337us/step - loss: 0.0062 - acc: 1.0000
Epoch 54/100
193/193 [==============================] - 0s 340us/step - loss: 0.0059 - acc: 1.0000
Epoch 55/100
193/193 [==============================] - 0s 335us/step - loss: 0.0056 - acc: 1.0000
Epoch 56/100
193/193 [==============================] - 0s 331us/step - loss: 0.0053 - acc: 1.0000
Epoch 57/100
193/193 [=

193/193 [==============================] - 0s 295us/step - loss: 0.0231 - acc: 1.0000
Epoch 30/100
193/193 [==============================] - 0s 290us/step - loss: 0.0216 - acc: 1.0000
Epoch 31/100
193/193 [==============================] - 0s 244us/step - loss: 0.0200 - acc: 1.0000
Epoch 32/100
193/193 [==============================] - 0s 238us/step - loss: 0.0186 - acc: 1.0000
Epoch 33/100
193/193 [==============================] - 0s 238us/step - loss: 0.0172 - acc: 1.0000
Epoch 34/100
193/193 [==============================] - 0s 249us/step - loss: 0.0162 - acc: 1.0000
Epoch 35/100
193/193 [==============================] - 0s 249us/step - loss: 0.0149 - acc: 1.0000
Epoch 36/100
193/193 [==============================] - 0s 233us/step - loss: 0.0140 - acc: 1.0000
Epoch 37/100
193/193 [==============================] - 0s 246us/step - loss: 0.0130 - acc: 1.0000
Epoch 38/100
193/193 [==============================] - 0s 254us/step - loss: 0.0126 - acc: 1.0000
Epoch 39/100
193/193 [=

...

195/195 [==============================] - 0s 426us/step - loss: 0.0074 - acc: 1.0000
Epoch 50/100
195/195 [==============================] - 0s 374us/step - loss: 0.0070 - acc: 1.0000
Epoch 51/100
195/195 [==============================] - 0s 416us/step - loss: 0.0066 - acc: 1.0000
Epoch 52/100
195/195 [==============================] - 0s 405us/step - loss: 0.0064 - acc: 1.0000
Epoch 53/100
195/195 [==============================] - 0s 380us/step - loss: 0.0060 - acc: 1.0000
Epoch 54/100
195/195 [==============================] - 0s 370us/step - loss: 0.0057 - acc: 1.0000
Epoch 55/100
195/195 [==============================] - 0s 385us/step - loss: 0.0055 - acc: 1.0000
Epoch 56/100
195/195 [==============================] - 0s 385us/step - loss: 0.0052 - acc: 1.0000
Epoch 57/100
195/195 [==============================] - 0s 385us/step - loss: 0.0050 - acc: 1.0000
Epoch 58/100
195/195 [==============================] - 0s 352us/step - loss: 0.0048 - acc: 1.0000
Epoch 59/100
195/195 [=

195/195 [==============================] - 0s 354us/step - loss: 0.0208 - acc: 1.0000
Epoch 32/100
195/195 [==============================] - 0s 364us/step - loss: 0.0193 - acc: 1.0000
Epoch 33/100
195/195 [==============================] - 0s 305us/step - loss: 0.0179 - acc: 1.0000
Epoch 34/100
195/195 [==============================] - 0s 359us/step - loss: 0.0169 - acc: 1.0000
Epoch 35/100
195/195 [==============================] - 0s 340us/step - loss: 0.0156 - acc: 1.0000
Epoch 36/100
195/195 [==============================] - 0s 324us/step - loss: 0.0145 - acc: 1.0000
Epoch 37/100
195/195 [==============================] - 0s 412us/step - loss: 0.0138 - acc: 1.0000
Epoch 38/100
195/195 [==============================] - 0s 383us/step - loss: 0.0130 - acc: 1.0000
Epoch 39/100
195/195 [==============================] - 0s 410us/step - loss: 0.0120 - acc: 1.0000
Epoch 40/100
195/195 [==============================] - 0s 354us/step - loss: 0.0113 - acc: 1.0000
Epoch 41/100
195/195 [=

195/195 [==============================] - 0s 498us/step - loss: 0.1259 - acc: 0.9795
Epoch 14/100
195/195 [==============================] - 0s 492us/step - loss: 0.1128 - acc: 0.9897
Epoch 15/100
195/195 [==============================] - 0s 308us/step - loss: 0.1000 - acc: 0.9949
Epoch 16/100
195/195 [==============================] - 0s 375us/step - loss: 0.0911 - acc: 0.9949
Epoch 17/100
195/195 [==============================] - 0s 418us/step - loss: 0.0816 - acc: 1.0000
Epoch 18/100
195/195 [==============================] - 0s 351us/step - loss: 0.0741 - acc: 1.0000
Epoch 19/100
195/195 [==============================] - 0s 496us/step - loss: 0.0667 - acc: 1.0000
Epoch 20/100
195/195 [==============================] - 0s 428us/step - loss: 0.0612 - acc: 1.0000
Epoch 21/100
195/195 [==============================] - 0s 426us/step - loss: 0.0567 - acc: 1.0000
Epoch 22/100
195/195 [==============================] - 0s 434us/step - loss: 0.0516 - acc: 1.0000
Epoch 23/100
195/195 [=

195/195 [==============================] - 0s 339us/step - loss: 0.0014 - acc: 1.0000
Epoch 96/100
195/195 [==============================] - 0s 308us/step - loss: 0.0014 - acc: 1.0000
Epoch 97/100
195/195 [==============================] - 0s 317us/step - loss: 0.0013 - acc: 1.0000
Epoch 98/100
195/195 [==============================] - 0s 315us/step - loss: 0.0013 - acc: 1.0000
Epoch 99/100
195/195 [==============================] - 0s 341us/step - loss: 0.0012 - acc: 1.0000
Epoch 100/100
13/13 [==============================] - 0s 32ms/step
Epoch 1/100
195/195 [==============================] - 2s 8ms/step - loss: 0.7137 - acc: 0.5231
Epoch 2/100
195/195 [==============================] - 0s 379us/step - loss: 0.4721 - acc: 0.7949
Epoch 3/100
195/195 [==============================] - 0s 371us/step - loss: 0.3846 - acc: 0.8564
Epoch 4/100
195/195 [==============================] - 0s 367us/step - loss: 0.3275 - acc: 0.9077
Epoch 5/100
195/195 [==============================] - 0s 43

195/195 [==============================] - 0s 395us/step - loss: 0.0021 - acc: 1.0000
Epoch 78/100
195/195 [==============================] - 0s 380us/step - loss: 0.0020 - acc: 1.0000
Epoch 79/100
195/195 [==============================] - 0s 313us/step - loss: 0.0019 - acc: 1.0000
Epoch 80/100
195/195 [==============================] - 0s 262us/step - loss: 0.0019 - acc: 1.0000
Epoch 81/100
195/195 [==============================] - 0s 241us/step - loss: 0.0018 - acc: 1.0000
Epoch 82/100
195/195 [==============================] - 0s 277us/step - loss: 0.0017 - acc: 1.0000
Epoch 83/100
195/195 [==============================] - 0s 277us/step - loss: 0.0017 - acc: 1.0000
Epoch 84/100
195/195 [==============================] - 0s 323us/step - loss: 0.0016 - acc: 1.0000
Epoch 85/100
195/195 [==============================] - 0s 323us/step - loss: 0.0016 - acc: 1.0000
Epoch 86/100
195/195 [==============================] - 0s 313us/step - loss: 0.0015 - acc: 1.0000
Epoch 87/100
195/195 [=

In [49]:
# smaller model
def create_baseline2():
    
	# create model, write code below
    model =Sequential()
    #model.add(Dense(60, activation='relu', input_shape=(60,)))
    model.add(Dense(30, activation='relu', input_shape=(60,)))
    model.add(Dense(1, activation='sigmoid'))

   # Compile model, write code below
    model.compile(optimizer='Adam',loss='binary_crossentropy',metrics=['accuracy'])
    return model

# evaluate model with standardized dataset
estimator = KerasClassifier(build_fn=create_baseline, epochs=100, batch_size=5, verbose=1)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(estimator, x, y, cv=kfold)
print("Results: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Epoch 1/100
186/186 [==============================] - 2s 11ms/step - loss: 0.7071 - acc: 0.4839
Epoch 2/100
186/186 [==============================] - 0s 699us/step - loss: 0.6854 - acc: 0.5538
Epoch 3/100
186/186 [==============================] - 0s 639us/step - loss: 0.6672 - acc: 0.6022
Epoch 4/100
186/186 [==============================] - 0s 615us/step - loss: 0.6527 - acc: 0.6075
Epoch 5/100
186/186 [==============================] - 0s 648us/step - loss: 0.6366 - acc: 0.6667
Epoch 6/100
186/186 [==============================] - 0s 705us/step - loss: 0.6209 - acc: 0.7312
Epoch 7/100
186/186 [==============================] - 0s 705us/step - loss: 0.6131 - acc: 0.6989
Epoch 8/100
186/186 [==============================] - 0s 764us/step - loss: 0.5969 - acc: 0.7581
Epoch 9/100
186/186 [==============================] - 0s 699us/step - loss: 0.5823 - acc: 0.7527
Epoch 10/100
186/186 [==============================] - 0s 554us/step - loss: 0.5696 - acc: 0.7903
Epoch 11/100
186/186

186/186 [==============================] - 0s 339us/step - loss: 0.2882 - acc: 0.8871
Epoch 84/100
186/186 [==============================] - 0s 350us/step - loss: 0.2861 - acc: 0.9086
Epoch 85/100
186/186 [==============================] - 0s 328us/step - loss: 0.2814 - acc: 0.9086
Epoch 86/100
186/186 [==============================] - 0s 350us/step - loss: 0.2866 - acc: 0.9032
Epoch 87/100
186/186 [==============================] - 0s 274us/step - loss: 0.2798 - acc: 0.8978
Epoch 88/100
186/186 [==============================] - 0s 274us/step - loss: 0.2847 - acc: 0.9032
Epoch 89/100
186/186 [==============================] - 0s 290us/step - loss: 0.2775 - acc: 0.8925
Epoch 90/100
186/186 [==============================] - 0s 311us/step - loss: 0.2803 - acc: 0.8871
Epoch 91/100
186/186 [==============================] - 0s 285us/step - loss: 0.2811 - acc: 0.8871
Epoch 92/100
186/186 [==============================] - 0s 402us/step - loss: 0.2708 - acc: 0.8978
Epoch 93/100
186/186 [=

187/187 [==============================] - 0s 326us/step - loss: 0.3361 - acc: 0.8396
Epoch 66/100
187/187 [==============================] - 0s 340us/step - loss: 0.3485 - acc: 0.8289
Epoch 67/100
187/187 [==============================] - 0s 337us/step - loss: 0.3366 - acc: 0.8396
Epoch 68/100
187/187 [==============================] - 0s 390us/step - loss: 0.3303 - acc: 0.8396
Epoch 69/100
187/187 [==============================] - 0s 401us/step - loss: 0.3429 - acc: 0.8235
Epoch 70/100
187/187 [==============================] - 0s 391us/step - loss: 0.3390 - acc: 0.8289
Epoch 71/100
187/187 [==============================] - 0s 353us/step - loss: 0.3336 - acc: 0.8503
Epoch 72/100
187/187 [==============================] - 0s 358us/step - loss: 0.3355 - acc: 0.8503
Epoch 73/100
187/187 [==============================] - 0s 274us/step - loss: 0.3254 - acc: 0.8610
Epoch 74/100
187/187 [==============================] - 0s 333us/step - loss: 0.3349 - acc: 0.8556
Epoch 75/100
187/187 [=

187/187 [==============================] - 0s 391us/step - loss: 0.3492 - acc: 0.8610
Epoch 48/100
187/187 [==============================] - 0s 369us/step - loss: 0.3577 - acc: 0.8503
Epoch 49/100
187/187 [==============================] - 0s 348us/step - loss: 0.3478 - acc: 0.8449
Epoch 50/100
187/187 [==============================] - 0s 353us/step - loss: 0.3419 - acc: 0.8556
Epoch 51/100
187/187 [==============================] - 0s 348us/step - loss: 0.3515 - acc: 0.8235
Epoch 52/100
187/187 [==============================] - 0s 358us/step - loss: 0.3402 - acc: 0.8610
Epoch 53/100
187/187 [==============================] - 0s 358us/step - loss: 0.3434 - acc: 0.8449
Epoch 54/100
187/187 [==============================] - 0s 352us/step - loss: 0.3497 - acc: 0.8556
Epoch 55/100
187/187 [==============================] - 0s 368us/step - loss: 0.3352 - acc: 0.8824
Epoch 56/100
187/187 [==============================] - 0s 349us/step - loss: 0.3412 - acc: 0.8503
Epoch 57/100
187/187 [=

187/187 [==============================] - 0s 481us/step - loss: 0.4022 - acc: 0.8342
Epoch 30/100
187/187 [==============================] - 0s 484us/step - loss: 0.3943 - acc: 0.8396
Epoch 31/100
187/187 [==============================] - 0s 440us/step - loss: 0.3845 - acc: 0.8556
Epoch 32/100
187/187 [==============================] - 0s 415us/step - loss: 0.3836 - acc: 0.8342
Epoch 33/100
187/187 [==============================] - 0s 420us/step - loss: 0.3851 - acc: 0.8396
Epoch 34/100
187/187 [==============================] - 0s 460us/step - loss: 0.3858 - acc: 0.8396
Epoch 35/100
187/187 [==============================] - 0s 424us/step - loss: 0.3724 - acc: 0.8342
Epoch 36/100
187/187 [==============================] - 0s 381us/step - loss: 0.3670 - acc: 0.8396
Epoch 37/100
187/187 [==============================] - 0s 379us/step - loss: 0.3639 - acc: 0.8449
Epoch 38/100
187/187 [==============================] - 0s 350us/step - loss: 0.3635 - acc: 0.8556
Epoch 39/100
187/187 [=

...

187/187 [==============================] - 0s 385us/step - loss: 0.3257 - acc: 0.8770
Epoch 58/100
187/187 [==============================] - 0s 407us/step - loss: 0.3261 - acc: 0.8663
Epoch 59/100
187/187 [==============================] - 0s 396us/step - loss: 0.3322 - acc: 0.8556
Epoch 60/100
187/187 [==============================] - 0s 380us/step - loss: 0.3224 - acc: 0.8717
Epoch 61/100
187/187 [==============================] - 0s 353us/step - loss: 0.3200 - acc: 0.8556
Epoch 62/100
187/187 [==============================] - 0s 350us/step - loss: 0.3184 - acc: 0.8610
Epoch 63/100
187/187 [==============================] - 0s 374us/step - loss: 0.3139 - acc: 0.8877
Epoch 64/100
187/187 [==============================] - 0s 369us/step - loss: 0.3065 - acc: 0.8770
Epoch 65/100
187/187 [==============================] - 0s 374us/step - loss: 0.3096 - acc: 0.8770
Epoch 66/100
187/187 [==============================] - 0s 407us/step - loss: 0.3023 - acc: 0.8824
Epoch 67/100
187/187 [=

188/188 [==============================] - 0s 383us/step - loss: 0.3476 - acc: 0.8511
Epoch 40/100
188/188 [==============================] - 0s 388us/step - loss: 0.3392 - acc: 0.8617
Epoch 41/100
188/188 [==============================] - 0s 404us/step - loss: 0.3364 - acc: 0.8617
Epoch 42/100
188/188 [==============================] - 0s 372us/step - loss: 0.3377 - acc: 0.8617
Epoch 43/100
188/188 [==============================] - 0s 388us/step - loss: 0.3279 - acc: 0.8670
Epoch 44/100
188/188 [==============================] - 0s 367us/step - loss: 0.3283 - acc: 0.8617
Epoch 45/100
188/188 [==============================] - 0s 357us/step - loss: 0.3311 - acc: 0.8617
Epoch 46/100
188/188 [==============================] - 0s 314us/step - loss: 0.3214 - acc: 0.8564
Epoch 47/100
188/188 [==============================] - 0s 319us/step - loss: 0.3225 - acc: 0.8670
Epoch 48/100
188/188 [==============================] - 0s 330us/step - loss: 0.3184 - acc: 0.8564
Epoch 49/100
188/188 [=

188/188 [==============================] - 0s 506us/step - loss: 0.4018 - acc: 0.8404
Epoch 22/100
188/188 [==============================] - 0s 500us/step - loss: 0.4020 - acc: 0.8457
Epoch 23/100
188/188 [==============================] - 0s 447us/step - loss: 0.3933 - acc: 0.8245
Epoch 24/100
188/188 [==============================] - 0s 431us/step - loss: 0.3830 - acc: 0.8404
Epoch 25/100
188/188 [==============================] - 0s 434us/step - loss: 0.3902 - acc: 0.8511
Epoch 26/100
188/188 [==============================] - 0s 401us/step - loss: 0.3815 - acc: 0.8511
Epoch 27/100
188/188 [==============================] - 0s 413us/step - loss: 0.3707 - acc: 0.8457
Epoch 28/100
188/188 [==============================] - 0s 484us/step - loss: 0.3610 - acc: 0.8457
Epoch 29/100
188/188 [==============================] - 0s 515us/step - loss: 0.3609 - acc: 0.8191
Epoch 30/100
188/188 [==============================] - 0s 483us/step - loss: 0.3633 - acc: 0.8457
Epoch 31/100
188/188 [=

188/188 [==============================] - 0s 777us/step - loss: 0.6500 - acc: 0.6117
Epoch 4/100
188/188 [==============================] - 0s 809us/step - loss: 0.6386 - acc: 0.7021
Epoch 5/100
188/188 [==============================] - 0s 671us/step - loss: 0.6244 - acc: 0.6915
Epoch 6/100
188/188 [==============================] - 0s 612us/step - loss: 0.6163 - acc: 0.6436
Epoch 7/100
188/188 [==============================] - 0s 555us/step - loss: 0.5954 - acc: 0.7500
Epoch 8/100
188/188 [==============================] - 0s 539us/step - loss: 0.5790 - acc: 0.7394
Epoch 9/100
188/188 [==============================] - 0s 566us/step - loss: 0.5623 - acc: 0.7606
Epoch 10/100
188/188 [==============================] - 0s 611us/step - loss: 0.5464 - acc: 0.7766
Epoch 11/100
188/188 [==============================] - 0s 654us/step - loss: 0.5292 - acc: 0.7926
Epoch 12/100
188/188 [==============================] - 0s 628us/step - loss: 0.5189 - acc: 0.7766
Epoch 13/100
188/188 [=======

Epoch 86/100
188/188 [==============================] - 0s 426us/step - loss: 0.2191 - acc: 0.9149
Epoch 87/100
188/188 [==============================] - 0s 367us/step - loss: 0.2188 - acc: 0.9362
Epoch 88/100
188/188 [==============================] - 0s 458us/step - loss: 0.2143 - acc: 0.9202
Epoch 89/100
188/188 [==============================] - 0s 426us/step - loss: 0.2146 - acc: 0.9202
Epoch 90/100
188/188 [==============================] - 0s 452us/step - loss: 0.2113 - acc: 0.9362
Epoch 91/100
188/188 [==============================] - 0s 420us/step - loss: 0.2080 - acc: 0.9362
Epoch 92/100
188/188 [==============================] - 0s 355us/step - loss: 0.2070 - acc: 0.9415
Epoch 93/100
188/188 [==============================] - 0s 431us/step - loss: 0.2020 - acc: 0.9468
Epoch 94/100
188/188 [==============================] - 0s 391us/step - loss: 0.2041 - acc: 0.9468
Epoch 95/100
188/188 [==============================] - 0s 371us/step - loss: 0.1972 - acc: 0.9521
Epoch 96/1

In [52]:
# evaluate baseline model with standardized dataset
np.random.seed(seed)
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasClassifier(build_fn=create_baseline, epochs=100, batch_size=5, verbose=1)))
pipeline = Pipeline(estimators)

kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(pipeline, x, y, cv=kfold)
print("Standardized: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Epoch 1/100
186/186 [==============================] - 4s 20ms/step - loss: 0.6276 - acc: 0.6505
Epoch 2/100
186/186 [==============================] - 0s 791us/step - loss: 0.5152 - acc: 0.7634
Epoch 3/100
186/186 [==============================] - 0s 650us/step - loss: 0.4427 - acc: 0.8333
Epoch 4/100
186/186 [==============================] - 0s 644us/step - loss: 0.3840 - acc: 0.8710
Epoch 5/100
186/186 [==============================] - 0s 546us/step - loss: 0.3398 - acc: 0.8978
Epoch 6/100
186/186 [==============================] - 0s 532us/step - loss: 0.3038 - acc: 0.9140
Epoch 7/100
186/186 [==============================] - 0s 521us/step - loss: 0.2703 - acc: 0.9355
Epoch 8/100
186/186 [==============================] - 0s 548us/step - loss: 0.2441 - acc: 0.9462
Epoch 9/100
186/186 [==============================] - 0s 714us/step - loss: 0.2231 - acc: 0.9516
Epoch 10/100
186/186 [==============================] - 0s 543us/step - loss: 0.2006 - acc: 0.9624
Epoch 11/100
186/186

186/186 [==============================] - 0s 463us/step - loss: 0.0034 - acc: 1.0000
Epoch 84/100
186/186 [==============================] - 0s 441us/step - loss: 0.0033 - acc: 1.0000
Epoch 85/100
186/186 [==============================] - 0s 420us/step - loss: 0.0032 - acc: 1.0000
Epoch 86/100
186/186 [==============================] - 0s 366us/step - loss: 0.0031 - acc: 1.0000
Epoch 87/100
186/186 [==============================] - 0s 307us/step - loss: 0.0030 - acc: 1.0000
Epoch 88/100
186/186 [==============================] - 0s 323us/step - loss: 0.0029 - acc: 1.0000
Epoch 89/100
186/186 [==============================] - 0s 320us/step - loss: 0.0028 - acc: 1.0000
Epoch 90/100
186/186 [==============================] - 0s 354us/step - loss: 0.0027 - acc: 1.0000
Epoch 91/100
186/186 [==============================] - 0s 345us/step - loss: 0.0027 - acc: 1.0000
Epoch 92/100
186/186 [==============================] - 0s 342us/step - loss: 0.0026 - acc: 1.0000
Epoch 93/100
186/186 [=

187/187 [==============================] - 0s 449us/step - loss: 0.0076 - acc: 1.0000
Epoch 66/100
187/187 [==============================] - 0s 492us/step - loss: 0.0073 - acc: 1.0000
Epoch 67/100
187/187 [==============================] - 0s 420us/step - loss: 0.0070 - acc: 1.0000
Epoch 68/100
187/187 [==============================] - 0s 415us/step - loss: 0.0067 - acc: 1.0000
Epoch 69/100
187/187 [==============================] - 0s 436us/step - loss: 0.0065 - acc: 1.0000
Epoch 70/100
187/187 [==============================] - 0s 487us/step - loss: 0.0062 - acc: 1.0000
Epoch 71/100
187/187 [==============================] - 0s 483us/step - loss: 0.0060 - acc: 1.0000
Epoch 72/100
187/187 [==============================] - 0s 475us/step - loss: 0.0058 - acc: 1.0000
Epoch 73/100
187/187 [==============================] - 0s 444us/step - loss: 0.0056 - acc: 1.0000
Epoch 74/100
187/187 [==============================] - 0s 420us/step - loss: 0.0053 - acc: 1.0000
Epoch 75/100
187/187 [=

187/187 [==============================] - 0s 978us/step - loss: 0.0159 - acc: 1.0000
Epoch 48/100
187/187 [==============================] - 0s 1ms/step - loss: 0.0151 - acc: 1.0000
Epoch 49/100
187/187 [==============================] - 0s 721us/step - loss: 0.0142 - acc: 1.0000
Epoch 50/100
187/187 [==============================] - 0s 668us/step - loss: 0.0135 - acc: 1.0000
Epoch 51/100
187/187 [==============================] - 0s 585us/step - loss: 0.0129 - acc: 1.0000
Epoch 52/100
187/187 [==============================] - 0s 634us/step - loss: 0.0122 - acc: 1.0000
Epoch 53/100
187/187 [==============================] - 0s 752us/step - loss: 0.0117 - acc: 1.0000
Epoch 54/100
187/187 [==============================] - 0s 668us/step - loss: 0.0111 - acc: 1.0000
Epoch 55/100
187/187 [==============================] - 0s 668us/step - loss: 0.0106 - acc: 1.0000
Epoch 56/100
187/187 [==============================] - 0s 752us/step - loss: 0.0102 - acc: 1.0000
Epoch 57/100
187/187 [===

187/187 [==============================] - 0s 786us/step - loss: 0.0562 - acc: 1.0000
Epoch 30/100
187/187 [==============================] - 0s 631us/step - loss: 0.0532 - acc: 1.0000
Epoch 31/100
187/187 [==============================] - 0s 615us/step - loss: 0.0496 - acc: 1.0000
Epoch 32/100
187/187 [==============================] - 0s 515us/step - loss: 0.0474 - acc: 1.0000
Epoch 33/100
187/187 [==============================] - 0s 449us/step - loss: 0.0434 - acc: 1.0000
Epoch 34/100
187/187 [==============================] - 0s 562us/step - loss: 0.0406 - acc: 1.0000
Epoch 35/100
187/187 [==============================] - 0s 777us/step - loss: 0.0382 - acc: 1.0000
Epoch 36/100
187/187 [==============================] - 0s 1ms/step - loss: 0.0363 - acc: 1.0000
Epoch 37/100
187/187 [==============================] - 0s 898us/step - loss: 0.0337 - acc: 1.0000
Epoch 38/100
187/187 [==============================] - 0s 919us/step - loss: 0.0324 - acc: 1.0000
Epoch 39/100
187/187 [===

...

187/187 [==============================] - 0s 867us/step - loss: 0.0117 - acc: 1.0000
Epoch 58/100
187/187 [==============================] - 0s 952us/step - loss: 0.0110 - acc: 1.0000
Epoch 59/100
187/187 [==============================] - 0s 867us/step - loss: 0.0105 - acc: 1.0000
Epoch 60/100
187/187 [==============================] - 0s 840us/step - loss: 0.0101 - acc: 1.0000
Epoch 61/100
187/187 [==============================] - 0s 775us/step - loss: 0.0097 - acc: 1.0000
Epoch 62/100
187/187 [==============================] - 0s 738us/step - loss: 0.0093 - acc: 1.0000
Epoch 63/100
187/187 [==============================] - 0s 825us/step - loss: 0.0089 - acc: 1.0000
Epoch 64/100
187/187 [==============================] - 0s 942us/step - loss: 0.0085 - acc: 1.0000
Epoch 65/100
187/187 [==============================] - 0s 872us/step - loss: 0.0081 - acc: 1.0000
Epoch 66/100
187/187 [==============================] - 0s 909us/step - loss: 0.0078 - acc: 1.0000
Epoch 67/100
187/187 [=

188/188 [==============================] - 0s 883us/step - loss: 0.0282 - acc: 1.0000
Epoch 40/100
188/188 [==============================] - 0s 905us/step - loss: 0.0267 - acc: 1.0000
Epoch 41/100
188/188 [==============================] - 0s 835us/step - loss: 0.0252 - acc: 1.0000
Epoch 42/100
188/188 [==============================] - 0s 842us/step - loss: 0.0238 - acc: 1.0000
Epoch 43/100
188/188 [==============================] - 0s 862us/step - loss: 0.0227 - acc: 1.0000
Epoch 44/100
188/188 [==============================] - 0s 926us/step - loss: 0.0216 - acc: 1.0000
Epoch 45/100
188/188 [==============================] - 0s 867us/step - loss: 0.0202 - acc: 1.0000
Epoch 46/100
188/188 [==============================] - 0s 910us/step - loss: 0.0193 - acc: 1.0000
Epoch 47/100
188/188 [==============================] - 0s 809us/step - loss: 0.0182 - acc: 1.0000
Epoch 48/100
188/188 [==============================] - 0s 814us/step - loss: 0.0173 - acc: 1.0000
Epoch 49/100
188/188 [=

188/188 [==============================] - 0s 835us/step - loss: 0.0867 - acc: 0.9947
Epoch 22/100
188/188 [==============================] - 0s 793us/step - loss: 0.0805 - acc: 0.9947
Epoch 23/100
188/188 [==============================] - 0s 764us/step - loss: 0.0736 - acc: 1.0000
Epoch 24/100
188/188 [==============================] - 0s 757us/step - loss: 0.0687 - acc: 1.0000
Epoch 25/100
188/188 [==============================] - 0s 963us/step - loss: 0.0644 - acc: 1.0000
Epoch 26/100
188/188 [==============================] - 0s 862us/step - loss: 0.0606 - acc: 1.0000
Epoch 27/100
188/188 [==============================] - 0s 761us/step - loss: 0.0555 - acc: 1.0000
Epoch 28/100
188/188 [==============================] - 0s 721us/step - loss: 0.0513 - acc: 1.0000
Epoch 29/100
188/188 [==============================] - 0s 764us/step - loss: 0.0482 - acc: 1.0000
Epoch 30/100
188/188 [==============================] - 0s 670us/step - loss: 0.0448 - acc: 1.0000
Epoch 31/100
188/188 [=

188/188 [==============================] - 0s 879us/step - loss: 0.4117 - acc: 0.8404
Epoch 4/100
188/188 [==============================] - 0s 921us/step - loss: 0.3600 - acc: 0.8723
Epoch 5/100
188/188 [==============================] - 0s 931us/step - loss: 0.3200 - acc: 0.9043
Epoch 6/100
188/188 [==============================] - 0s 819us/step - loss: 0.2885 - acc: 0.9096
Epoch 7/100
188/188 [==============================] - 0s 841us/step - loss: 0.2606 - acc: 0.9149
Epoch 8/100
188/188 [==============================] - 0s 921us/step - loss: 0.2357 - acc: 0.9362
Epoch 9/100
188/188 [==============================] - 0s 857us/step - loss: 0.2125 - acc: 0.9468
Epoch 10/100
188/188 [==============================] - 0s 910us/step - loss: 0.1962 - acc: 0.9628
Epoch 11/100
188/188 [==============================] - 0s 1ms/step - loss: 0.1767 - acc: 0.9734
Epoch 12/100
188/188 [==============================] - 0s 878us/step - loss: 0.1619 - acc: 0.9787
Epoch 13/100
188/188 [=========

188/188 [==============================] - 0s 873us/step - loss: 0.0034 - acc: 1.0000
Epoch 87/100
188/188 [==============================] - 0s 772us/step - loss: 0.0032 - acc: 1.0000
Epoch 88/100
188/188 [==============================] - 0s 781us/step - loss: 0.0031 - acc: 1.0000
Epoch 89/100
188/188 [==============================] - 0s 979us/step - loss: 0.0030 - acc: 1.0000
Epoch 90/100
188/188 [==============================] - 0s 809us/step - loss: 0.0029 - acc: 1.0000
Epoch 91/100
188/188 [==============================] - 0s 756us/step - loss: 0.0028 - acc: 1.0000
Epoch 92/100
188/188 [==============================] - 0s 708us/step - loss: 0.0027 - acc: 1.0000
Epoch 93/100
188/188 [==============================] - 0s 662us/step - loss: 0.0027 - acc: 1.0000
Epoch 94/100
188/188 [==============================] - 0s 694us/step - loss: 0.0026 - acc: 1.0000
Epoch 95/100
188/188 [==============================] - 0s 806us/step - loss: 0.0025 - acc: 1.0000
Epoch 96/100
188/188 [=

In [53]:
# large model
def create_larger():
    
	# create model, write code below
    model =Sequential()
    model.add(Dense(60, activation='relu', input_shape=(60,)))
    model.add(Dense(30, activation='relu', input_shape=(60,)))   
    model.add(Dense(1, activation='sigmoid'))

	# Compile model, write code below
    model.compile(optimizer='Adam',loss='binary_crossentropy',metrics=['accuracy'])
	
    return model
# evaluate model with standardized dataset
estimator = KerasClassifier(build_fn=create_larger, epochs=100, batch_size=5, verbose=1)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(estimator, x, y, cv=kfold)
print("Results: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Epoch 1/100
186/186 [==============================] - 4s 22ms/step - loss: 0.7184 - acc: 0.4731
Epoch 2/100
186/186 [==============================] - 0s 1ms/step - loss: 0.6622 - acc: 0.6129
Epoch 3/100
186/186 [==============================] - 0s 1ms/step - loss: 0.6274 - acc: 0.7043
Epoch 4/100
186/186 [==============================] - 0s 1ms/step - loss: 0.5882 - acc: 0.7366
Epoch 5/100
186/186 [==============================] - 0s 1ms/step - loss: 0.5652 - acc: 0.7419
Epoch 6/100
186/186 [==============================] - 0s 941us/step - loss: 0.5279 - acc: 0.7742
Epoch 7/100
186/186 [==============================] - 0s 1ms/step - loss: 0.4905 - acc: 0.7796
Epoch 8/100
186/186 [==============================] - 0s 963us/step - loss: 0.4643 - acc: 0.7957
Epoch 9/100
186/186 [==============================] - 0s 797us/step - loss: 0.4680 - acc: 0.8065
Epoch 10/100
186/186 [==============================] - 0s 843us/step - loss: 0.4229 - acc: 0.8118
Epoch 11/100
186/186 [========

Epoch 84/100
186/186 [==============================] - 0s 489us/step - loss: 0.0071 - acc: 1.0000
Epoch 85/100
186/186 [==============================] - 0s 450us/step - loss: 0.0068 - acc: 1.0000
Epoch 86/100
186/186 [==============================] - 0s 409us/step - loss: 0.0065 - acc: 1.0000
Epoch 87/100
186/186 [==============================] - 0s 388us/step - loss: 0.0059 - acc: 1.0000
Epoch 88/100
186/186 [==============================] - 0s 379us/step - loss: 0.0058 - acc: 1.0000
Epoch 89/100
186/186 [==============================] - 0s 399us/step - loss: 0.0051 - acc: 1.0000
Epoch 90/100
186/186 [==============================] - 0s 671us/step - loss: 0.0049 - acc: 1.0000
Epoch 91/100
186/186 [==============================] - 0s 1ms/step - loss: 0.0045 - acc: 1.0000
Epoch 92/100
186/186 [==============================] - 0s 1ms/step - loss: 0.0048 - acc: 1.0000
Epoch 93/100
186/186 [==============================] - 0s 1ms/step - loss: 0.0042 - acc: 1.0000
Epoch 94/100
186

187/187 [==============================] - 0s 940us/step - loss: 0.0506 - acc: 0.9947
Epoch 67/100
187/187 [==============================] - 0s 836us/step - loss: 0.0504 - acc: 0.9947
Epoch 68/100
187/187 [==============================] - 0s 919us/step - loss: 0.0435 - acc: 0.9947
Epoch 69/100
187/187 [==============================] - 0s 836us/step - loss: 0.0367 - acc: 0.9947
Epoch 70/100
187/187 [==============================] - 0s 1ms/step - loss: 0.0390 - acc: 1.0000
Epoch 71/100
187/187 [==============================] - 0s 919us/step - loss: 0.0373 - acc: 1.0000
Epoch 72/100
187/187 [==============================] - 0s 915us/step - loss: 0.0345 - acc: 0.9947
Epoch 73/100
187/187 [==============================] - 0s 836us/step - loss: 0.0319 - acc: 1.0000
Epoch 74/100
187/187 [==============================] - 0s 836us/step - loss: 0.0284 - acc: 1.0000
Epoch 75/100
187/187 [==============================] - 0s 919us/step - loss: 0.0351 - acc: 1.0000
Epoch 76/100
187/187 [===

187/187 [==============================] - 0s 836us/step - loss: 0.0841 - acc: 0.9786
Epoch 49/100
187/187 [==============================] - 0s 822us/step - loss: 0.0840 - acc: 0.9840
Epoch 50/100
187/187 [==============================] - 0s 919us/step - loss: 0.0737 - acc: 0.9893
Epoch 51/100
187/187 [==============================] - 0s 1ms/step - loss: 0.0653 - acc: 0.9947
Epoch 52/100
187/187 [==============================] - 0s 836us/step - loss: 0.0651 - acc: 1.0000
Epoch 53/100
187/187 [==============================] - 0s 836us/step - loss: 0.0642 - acc: 0.9893
Epoch 54/100
187/187 [==============================] - 0s 826us/step - loss: 0.0553 - acc: 0.9947
Epoch 55/100
187/187 [==============================] - 0s 836us/step - loss: 0.0523 - acc: 0.9947
Epoch 56/100
187/187 [==============================] - 0s 919us/step - loss: 0.0559 - acc: 0.9947
Epoch 57/100
187/187 [==============================] - 0s 1ms/step - loss: 0.0456 - acc: 1.0000
Epoch 58/100
187/187 [=====

187/187 [==============================] - 0s 927us/step - loss: 0.2242 - acc: 0.9144
Epoch 31/100
187/187 [==============================] - 0s 919us/step - loss: 0.1983 - acc: 0.9305
Epoch 32/100
187/187 [==============================] - 0s 836us/step - loss: 0.1823 - acc: 0.9251
Epoch 33/100
187/187 [==============================] - 0s 919us/step - loss: 0.2225 - acc: 0.8984
Epoch 34/100
187/187 [==============================] - 0s 919us/step - loss: 0.2027 - acc: 0.9091
Epoch 35/100
187/187 [==============================] - 0s 1ms/step - loss: 0.1731 - acc: 0.9198
Epoch 36/100
187/187 [==============================] - 0s 836us/step - loss: 0.1605 - acc: 0.9465
Epoch 37/100
187/187 [==============================] - 0s 752us/step - loss: 0.1531 - acc: 0.9465
Epoch 38/100
187/187 [==============================] - 0s 668us/step - loss: 0.1524 - acc: 0.9412
Epoch 39/100
187/187 [==============================] - 0s 919us/step - loss: 0.1448 - acc: 0.9626
Epoch 40/100
187/187 [===

...

187/187 [==============================] - 0s 604us/step - loss: 0.1080 - acc: 0.9626
Epoch 61/100
187/187 [==============================] - 0s 563us/step - loss: 0.0790 - acc: 0.9840
Epoch 62/100
187/187 [==============================] - 0s 607us/step - loss: 0.0839 - acc: 0.9786
Epoch 63/100
187/187 [==============================] - 0s 637us/step - loss: 0.0728 - acc: 0.9893
Epoch 64/100
187/187 [==============================] - 0s 557us/step - loss: 0.0784 - acc: 0.9893
Epoch 65/100
187/187 [==============================] - 0s 533us/step - loss: 0.0558 - acc: 0.9947
Epoch 66/100
187/187 [==============================] - 0s 525us/step - loss: 0.0813 - acc: 0.9786
Epoch 67/100
187/187 [==============================] - 0s 645us/step - loss: 0.0861 - acc: 0.9840
Epoch 68/100
187/187 [==============================] - 0s 770us/step - loss: 0.0734 - acc: 0.9893
Epoch 69/100
187/187 [==============================] - 0s 684us/step - loss: 0.0476 - acc: 0.9947
Epoch 70/100
187/187 [=

188/188 [==============================] - 0s 914us/step - loss: 0.1384 - acc: 0.9521
Epoch 43/100
188/188 [==============================] - 0s 914us/step - loss: 0.1246 - acc: 0.9574
Epoch 44/100
188/188 [==============================] - 0s 914us/step - loss: 0.1368 - acc: 0.9521
Epoch 45/100
188/188 [==============================] - 0s 1ms/step - loss: 0.1085 - acc: 0.9840
Epoch 46/100
188/188 [==============================] - 0s 1ms/step - loss: 0.1031 - acc: 0.9787
Epoch 47/100
188/188 [==============================] - 0s 905us/step - loss: 0.1131 - acc: 0.9628
Epoch 48/100
188/188 [==============================] - 0s 1ms/step - loss: 0.0998 - acc: 0.9894
Epoch 49/100
188/188 [==============================] - 0s 997us/step - loss: 0.1191 - acc: 0.9574
Epoch 50/100
188/188 [==============================] - 0s 1ms/step - loss: 0.0917 - acc: 0.9894
Epoch 51/100
188/188 [==============================] - 0s 1ms/step - loss: 0.0855 - acc: 0.9894
Epoch 52/100
188/188 [===========

188/188 [==============================] - 0s 750us/step - loss: 0.2615 - acc: 0.8723
Epoch 25/100
188/188 [==============================] - 0s 702us/step - loss: 0.2588 - acc: 0.8883
Epoch 26/100
188/188 [==============================] - 0s 654us/step - loss: 0.2605 - acc: 0.8989
Epoch 27/100
188/188 [==============================] - 0s 606us/step - loss: 0.2421 - acc: 0.8723
Epoch 28/100
188/188 [==============================] - 0s 578us/step - loss: 0.2346 - acc: 0.8936
Epoch 29/100
188/188 [==============================] - 0s 621us/step - loss: 0.2270 - acc: 0.8936
Epoch 30/100
188/188 [==============================] - 0s 724us/step - loss: 0.2132 - acc: 0.9149
Epoch 31/100
188/188 [==============================] - 0s 622us/step - loss: 0.1986 - acc: 0.9202 0s - loss: 0.2281 - acc: 0.89
Epoch 32/100
188/188 [==============================] - 0s 717us/step - loss: 0.2369 - acc: 0.8883
Epoch 33/100
188/188 [==============================] - 0s 772us/step - loss: 0.2118 - acc: 

188/188 [==============================] - 0s 835us/step - loss: 0.5135 - acc: 0.7500
Epoch 7/100
188/188 [==============================] - 0s 830us/step - loss: 0.4850 - acc: 0.7766
Epoch 8/100
188/188 [==============================] - 0s 883us/step - loss: 0.4425 - acc: 0.8138
Epoch 9/100
188/188 [==============================] - 0s 889us/step - loss: 0.4276 - acc: 0.8032
Epoch 10/100
188/188 [==============================] - 0s 835us/step - loss: 0.4319 - acc: 0.8032
Epoch 11/100
188/188 [==============================] - 0s 780us/step - loss: 0.4073 - acc: 0.8191
Epoch 12/100
188/188 [==============================] - 0s 716us/step - loss: 0.3944 - acc: 0.8245
Epoch 13/100
188/188 [==============================] - 0s 721us/step - loss: 0.3693 - acc: 0.8404
Epoch 14/100
188/188 [==============================] - 0s 839us/step - loss: 0.3627 - acc: 0.8511
Epoch 15/100
188/188 [==============================] - 0s 907us/step - loss: 0.3425 - acc: 0.8511
Epoch 16/100
188/188 [====

Epoch 89/100
188/188 [==============================] - 0s 517us/step - loss: 0.0090 - acc: 1.0000
Epoch 90/100
188/188 [==============================] - 0s 434us/step - loss: 0.0097 - acc: 1.0000
Epoch 91/100
188/188 [==============================] - 0s 355us/step - loss: 0.0084 - acc: 1.0000
Epoch 92/100
188/188 [==============================] - 0s 438us/step - loss: 0.0086 - acc: 1.0000
Epoch 93/100
188/188 [==============================] - 0s 428us/step - loss: 0.0094 - acc: 1.0000
Epoch 94/100
188/188 [==============================] - 0s 500us/step - loss: 0.0090 - acc: 1.0000
Epoch 95/100
188/188 [==============================] - 0s 1ms/step - loss: 0.0075 - acc: 1.0000
Epoch 96/100
188/188 [==============================] - 0s 1ms/step - loss: 0.0067 - acc: 1.0000
Epoch 97/100
188/188 [==============================] - 0s 1ms/step - loss: 0.0068 - acc: 1.0000
Epoch 98/100
188/188 [==============================] - 0s 1ms/step - loss: 0.0061 - acc: 1.0000
Epoch 99/100
188/1

In [54]:
# evaluate baseline model with standardized dataset
np.random.seed(seed)
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasClassifier(build_fn=create_larger, epochs=100, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)

kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(pipeline, x, y, cv=kfold)
print("Larger: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Larger: 84.61% (6.36%)


In [55]:
# overfitting model
def create_overfit():
    
	# create model, write code below
    model =Sequential()
    model.add(Dense(120, activation='relu', input_shape=(60,)))
    model.add(Dense(90, activation='relu'))
    model.add(Dense(60, activation='relu'))
    model.add(Dense(30, activation='relu'))    
    model.add(Dense(1, activation='sigmoid'))

	# Compile model, write code below
    model.compile(optimizer='Adam',loss='binary_crossentropy',metrics=['accuracy'])
	
    return model
# evaluate model with standardized dataset
estimator = KerasClassifier(build_fn=create_overfit, epochs=100, batch_size=5, verbose=1)
kfold = StratifiedKFold(n_splits=15, shuffle=True, random_state=seed)
results = cross_val_score(estimator, x, y, cv=kfold)
print("Overfit Results: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Epoch 1/100
193/193 [==============================] - 7s 34ms/step - loss: 0.6844 - acc: 0.5648
Epoch 2/100
193/193 [==============================] - 0s 983us/step - loss: 0.6245 - acc: 0.6839
Epoch 3/100
193/193 [==============================] - 0s 901us/step - loss: 0.5362 - acc: 0.7409
Epoch 4/100
193/193 [==============================] - 0s 1ms/step - loss: 0.5010 - acc: 0.7668
Epoch 5/100
193/193 [==============================] - 0s 1ms/step - loss: 0.4870 - acc: 0.7668
Epoch 6/100
193/193 [==============================] - 0s 1ms/step - loss: 0.4153 - acc: 0.8135
Epoch 7/100
193/193 [==============================] - 0s 1ms/step - loss: 0.3721 - acc: 0.8446
Epoch 8/100
193/193 [==============================] - 0s 886us/step - loss: 0.3242 - acc: 0.8549
Epoch 9/100
193/193 [==============================] - 0s 938us/step - loss: 0.3044 - acc: 0.8497
Epoch 10/100
193/193 [==============================] - 0s 1ms/step - loss: 0.2824 - acc: 0.9016
Epoch 11/100
193/193 [========

193/193 [==============================] - 0s 1ms/step - loss: 3.7642e-05 - acc: 1.0000
Epoch 83/100
193/193 [==============================] - 0s 1ms/step - loss: 3.6507e-05 - acc: 1.0000
Epoch 84/100
193/193 [==============================] - 0s 1ms/step - loss: 3.4799e-05 - acc: 1.0000
Epoch 85/100
193/193 [==============================] - 0s 2ms/step - loss: 3.3270e-05 - acc: 1.0000
Epoch 86/100
193/193 [==============================] - 0s 1ms/step - loss: 3.2111e-05 - acc: 1.0000
Epoch 87/100
193/193 [==============================] - 0s 1ms/step - loss: 3.0745e-05 - acc: 1.0000
Epoch 88/100
193/193 [==============================] - 0s 1ms/step - loss: 2.9567e-05 - acc: 1.0000
Epoch 89/100
193/193 [==============================] - 0s 1ms/step - loss: 2.8402e-05 - acc: 1.0000
Epoch 90/100
193/193 [==============================] - 0s 1ms/step - loss: 2.7554e-05 - acc: 1.0000
Epoch 91/100
193/193 [==============================] - 0s 1ms/step - loss: 2.5900e-05 - acc: 1.0000
Epo

Epoch 63/100
193/193 [==============================] - 0s 741us/step - loss: 9.2960e-05 - acc: 1.0000
Epoch 64/100
193/193 [==============================] - 0s 789us/step - loss: 8.7459e-05 - acc: 1.0000
Epoch 65/100
193/193 [==============================] - 0s 764us/step - loss: 8.3491e-05 - acc: 1.0000
Epoch 66/100
193/193 [==============================] - 0s 779us/step - loss: 7.9040e-05 - acc: 1.0000
Epoch 67/100
193/193 [==============================] - 0s 626us/step - loss: 7.5793e-05 - acc: 1.0000
Epoch 68/100
193/193 [==============================] - 0s 689us/step - loss: 7.1788e-05 - acc: 1.0000
Epoch 69/100
193/193 [==============================] - 0s 680us/step - loss: 6.8799e-05 - acc: 1.0000
Epoch 70/100
193/193 [==============================] - 0s 668us/step - loss: 6.5418e-05 - acc: 1.0000
Epoch 71/100
193/193 [==============================] - 0s 798us/step - loss: 6.2430e-05 - acc: 1.0000
Epoch 72/100
193/193 [==============================] - 0s 783us/step - l

193/193 [==============================] - 0s 739us/step - loss: 9.5550e-04 - acc: 1.0000
Epoch 45/100
193/193 [==============================] - 0s 806us/step - loss: 9.8333e-04 - acc: 1.0000
Epoch 46/100
193/193 [==============================] - 0s 821us/step - loss: 8.5180e-04 - acc: 1.0000
Epoch 47/100
193/193 [==============================] - 0s 2ms/step - loss: 8.0156e-04 - acc: 1.0000
Epoch 48/100
193/193 [==============================] - 0s 1ms/step - loss: 7.7550e-04 - acc: 1.0000
Epoch 49/100
193/193 [==============================] - 0s 2ms/step - loss: 6.5449e-04 - acc: 1.0000
Epoch 50/100
193/193 [==============================] - 0s 1ms/step - loss: 6.2632e-04 - acc: 1.0000
Epoch 51/100
193/193 [==============================] - 0s 1ms/step - loss: 6.0338e-04 - acc: 1.0000
Epoch 52/100
193/193 [==============================] - 0s 1ms/step - loss: 5.2737e-04 - acc: 1.0000
Epoch 53/100
193/193 [==============================] - 0s 1ms/step - loss: 5.0740e-04 - acc: 1.00

Epoch 25/100
193/193 [==============================] - 0s 728us/step - loss: 0.0406 - acc: 0.9948
Epoch 26/100
193/193 [==============================] - 0s 764us/step - loss: 0.1404 - acc: 0.9585
Epoch 27/100
193/193 [==============================] - 0s 730us/step - loss: 0.0715 - acc: 0.9637
Epoch 28/100
193/193 [==============================] - 0s 2ms/step - loss: 0.0812 - acc: 0.9689
Epoch 29/100
193/193 [==============================] - 0s 1ms/step - loss: 0.0530 - acc: 0.9793
Epoch 30/100
193/193 [==============================] - 0s 1ms/step - loss: 0.0352 - acc: 0.9896
Epoch 31/100
193/193 [==============================] - 0s 1ms/step - loss: 0.0145 - acc: 1.0000
Epoch 32/100
193/193 [==============================] - 0s 2ms/step - loss: 0.0070 - acc: 1.0000
Epoch 33/100
193/193 [==============================] - 0s 1ms/step - loss: 0.0036 - acc: 1.0000
Epoch 34/100
193/193 [==============================] - 0s 1ms/step - loss: 0.0029 - acc: 1.0000
Epoch 35/100
193/193 [==

...

195/195 [==============================] - 0s 1ms/step - loss: 0.0013 - acc: 1.0000
Epoch 37/100
195/195 [==============================] - 0s 2ms/step - loss: 0.0013 - acc: 1.0000
Epoch 38/100
195/195 [==============================] - 0s 2ms/step - loss: 0.0012 - acc: 1.0000
Epoch 39/100
195/195 [==============================] - 0s 1ms/step - loss: 9.7581e-04 - acc: 1.0000
Epoch 40/100
195/195 [==============================] - 0s 1ms/step - loss: 8.5896e-04 - acc: 1.0000
Epoch 41/100
195/195 [==============================] - 0s 1ms/step - loss: 7.7458e-04 - acc: 1.0000
Epoch 42/100
195/195 [==============================] - 0s 1ms/step - loss: 6.7413e-04 - acc: 1.0000
Epoch 43/100
195/195 [==============================] - 0s 1ms/step - loss: 6.4684e-04 - acc: 1.0000
Epoch 44/100
195/195 [==============================] - 0s 1ms/step - loss: 5.6934e-04 - acc: 1.0000
Epoch 45/100
195/195 [==============================] - 0s 1ms/step - loss: 5.4127e-04 - acc: 1.0000
Epoch 46/100
19

195/195 [==============================] - 0s 2ms/step - loss: 0.1146 - acc: 0.9538
Epoch 18/100
195/195 [==============================] - 0s 1ms/step - loss: 0.0869 - acc: 0.9590
Epoch 19/100
195/195 [==============================] - 0s 1ms/step - loss: 0.1464 - acc: 0.9538
Epoch 20/100
195/195 [==============================] - 0s 1ms/step - loss: 0.0859 - acc: 0.9692
Epoch 21/100
195/195 [==============================] - 0s 1ms/step - loss: 0.1007 - acc: 0.9641
Epoch 22/100
195/195 [==============================] - 0s 1ms/step - loss: 0.0721 - acc: 0.9795
Epoch 23/100
195/195 [==============================] - 0s 1ms/step - loss: 0.0319 - acc: 0.9949
Epoch 24/100
195/195 [==============================] - 0s 1ms/step - loss: 0.0143 - acc: 1.0000
Epoch 25/100
195/195 [==============================] - 0s 1ms/step - loss: 0.0118 - acc: 1.0000
Epoch 26/100
195/195 [==============================] - 0s 1ms/step - loss: 0.0886 - acc: 0.9641
Epoch 27/100
195/195 [=====================

195/195 [==============================] - 0s 910us/step - loss: 5.5360e-05 - acc: 1.0000
Epoch 99/100
195/195 [==============================] - 0s 1ms/step - loss: 5.4276e-05 - acc: 1.0000
Epoch 100/100
13/13 [==============================] - 2s 167ms/step
Epoch 1/100
195/195 [==============================] - 5s 27ms/step - loss: 0.6793 - acc: 0.5538
Epoch 2/100
195/195 [==============================] - 0s 1ms/step - loss: 0.6337 - acc: 0.6564
Epoch 3/100
195/195 [==============================] - 0s 2ms/step - loss: 0.5478 - acc: 0.7487
Epoch 4/100
195/195 [==============================] - 0s 1ms/step - loss: 0.5018 - acc: 0.7282
Epoch 5/100
195/195 [==============================] - 0s 1ms/step - loss: 0.4385 - acc: 0.7692
Epoch 6/100
195/195 [==============================] - 0s 1ms/step - loss: 0.3952 - acc: 0.8205
Epoch 7/100
195/195 [==============================] - 0s 1ms/step - loss: 0.4032 - acc: 0.8000
Epoch 8/100
195/195 [==============================] - 0s 1ms/step 

195/195 [==============================] - 0s 957us/step - loss: 1.3979e-04 - acc: 1.0000
Epoch 81/100
195/195 [==============================] - 0s 1ms/step - loss: 1.3291e-04 - acc: 1.0000
Epoch 82/100
195/195 [==============================] - 0s 1ms/step - loss: 1.2699e-04 - acc: 1.0000
Epoch 83/100
195/195 [==============================] - 0s 933us/step - loss: 1.2129e-04 - acc: 1.0000
Epoch 84/100
195/195 [==============================] - 0s 1ms/step - loss: 1.1609e-04 - acc: 1.0000
Epoch 85/100
195/195 [==============================] - 0s 869us/step - loss: 1.1236e-04 - acc: 1.0000
Epoch 86/100
195/195 [==============================] - 0s 2ms/step - loss: 1.0803e-04 - acc: 1.0000
Epoch 87/100
195/195 [==============================] - 0s 2ms/step - loss: 1.0434e-04 - acc: 1.0000
Epoch 88/100
195/195 [==============================] - 0s 2ms/step - loss: 1.0114e-04 - acc: 1.0000
Epoch 89/100
195/195 [==============================] - 0s 2ms/step - loss: 9.6641e-05 - acc: 1.00

Epoch 62/100
195/195 [==============================] - 0s 921us/step - loss: 2.1049e-04 - acc: 1.0000
Epoch 63/100
195/195 [==============================] - 0s 847us/step - loss: 1.9880e-04 - acc: 1.0000
Epoch 64/100
195/195 [==============================] - 0s 1ms/step - loss: 1.9977e-04 - acc: 1.0000
Epoch 65/100
195/195 [==============================] - 0s 792us/step - loss: 1.7793e-04 - acc: 1.0000
Epoch 66/100
195/195 [==============================] - 0s 794us/step - loss: 1.6795e-04 - acc: 1.0000
Epoch 67/100
195/195 [==============================] - 0s 2ms/step - loss: 1.6579e-04 - acc: 1.0000
Epoch 68/100
195/195 [==============================] - 0s 2ms/step - loss: 1.5202e-04 - acc: 1.0000
Epoch 69/100
195/195 [==============================] - 0s 2ms/step - loss: 1.4689e-04 - acc: 1.0000
Epoch 70/100
195/195 [==============================] - 0s 2ms/step - loss: 1.4039e-04 - acc: 1.0000
Epoch 71/100
195/195 [==============================] - 0s 1ms/step - loss: 1.3170e